In [ ]:
!pip install -q kaggle
from google.colab import drive
drive.mount('/content/gdrive')
 
!pip3 install --upgrade --force-reinstall --no-deps kaggle
 
!mkdir ~/.kaggle  
!cp ./gdrive/MyDrive/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c ubiquant-market-prediction

In [ ]:
!unzip ubiquant-market-prediction.zip

In [ ]:
import sklearn as sk
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp
import tensorflow_hub as hub
import os

In [ ]:
tf.debugging.set_log_device_placement(False)
gpus = tf.config.list_logical_devices('GPU')
strategy = tf.distribute.MirroredStrategy(gpus)

In [ ]:
with open('train.csv') as f:
  print(f.readline())
  print(len(f.readline().split(',')))

In [ ]:
column_names = ['row_id', 'time_id', 'investment_id', 'target']
column_names.extend(['f_' + str(i) for i in range(300)])

In [ ]:
types = [tf.string, tf.int64, tf.int64]
types.extend([tf.float64] * 301)

In [ ]:
train_x = tf.data.experimental.make_csv_dataset('train.csv', batch_size=32, column_defaults=types[4::], select_columns=column_names[4::])
train_y = tf.data.experimental.make_csv_dataset('train.csv', batch_size=32, column_defaults=types[3:4:], select_columns=column_names[3:4:])

In [ ]:
train = tf.data.Dataset.zip((train_x, train_y))

In [ ]:
def get_model():
 with strategy.scope():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Input(shape=(300,)))
    model.add(tf.keras.layers.Dense(100, activation=tf.nn.elu))
    model.add(tf.keras.layers.Dropout(0.1))
    model.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid))
    model.compile(optimizer='Adam', loss='MSLE', metrics=[tf.keras.metrics.RootMeanSquaredError()])
    return model

In [ ]:
model = get_model()

In [ ]:
model.fit(x=train, epochs=1, batch_size=32)